# 🌲 松材线虫病知识图谱构建 - Google Colab版

**运行环境**: Google Colab (免费GPU)

**预计时间**: 37.5分钟 (150块) 或 25分钟 (100块)

**优势**:
- ✅ 完全免费
- ✅ 无需本地计算资源
- ✅ 自动保存结果到Google Drive
- ✅ 可以关闭浏览器，任务继续运行

---

## 📋 使用步骤

1. **挂载Google Drive** - 运行第1个单元格
2. **上传文献** - 将PDF文件上传到Drive
3. **安装依赖** - 运行安装单元格
4. **运行任务** - 执行主程序
5. **下载结果** - 从Drive下载结果文件

---

## 1️⃣ 挂载Google Drive

In [ ]:
from google.colab import drive
import os

# 挂载Google Drive
drive.mount('/content/drive')

# 创建工作目录
WORK_DIR = '/content/drive/MyDrive/PWD_KG'
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

print(f"✓ 工作目录: {WORK_DIR}")
print(f"✓ 当前目录: {os.getcwd()}")

## 2️⃣ 上传项目文件

### 方法A: 从GitHub克隆 (推荐)

In [ ]:
# 如果您的项目在GitHub上
# !git clone https://github.com/your-username/PWD.git .

# 或者手动上传文件到 /content/drive/MyDrive/PWD_KG/
print("请将以下文件上传到Google Drive的 PWD_KG 文件夹:")
print("  - 所有 .py 文件")
print("  - config/ 文件夹")
print("  - requirements.txt")
print("  - 文献/ 文件夹 (包含PDF文件)")

### 方法B: 直接上传文件

In [ ]:
from google.colab import files
import zipfile

# 上传项目压缩包
print("请上传项目压缩包 (PWD.zip)")
uploaded = files.upload()

# 解压
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✓ 已解压: {filename}")

!ls -la

## 3️⃣ 安装依赖

In [ ]:
%%bash
# 安装Ollama
echo "安装Ollama..."
curl -fsSL https://ollama.ai/install.sh | sh

# 启动Ollama服务
echo "启动Ollama服务..."
nohup ollama serve > /tmp/ollama.log 2>&1 &
sleep 5

echo "✓ Ollama安装完成"

In [ ]:
# 拉取模型 (约1-2分钟)
!ollama pull llama3.2:3b

print("✓ 模型下载完成")

In [ ]:
# 安装Python依赖
!pip install -q -r requirements.txt

print("✓ Python依赖安装完成")

## 4️⃣ 配置参数 (可选)

In [ ]:
# 快速配置
MAX_CHUNKS = 100  # 100块约25分钟, 150块约37.5分钟

# 修改配置文件
import yaml

config_path = 'config/config.yaml'
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

config['llm']['max_chunks'] = MAX_CHUNKS

with open(config_path, 'w', encoding='utf-8') as f:
    yaml.dump(config, f, allow_unicode=True)

print(f"✓ 配置已更新: max_chunks = {MAX_CHUNKS}")
print(f"  预计时间: {MAX_CHUNKS * 0.25:.1f} 分钟")

## 5️⃣ 运行知识图谱构建

**注意**: 运行期间可以关闭浏览器标签页，Colab会继续运行

In [ ]:
import sys
import logging
from datetime import datetime

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

print("="*60)
print("🚀 开始构建知识图谱")
print(f"⏰ 开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

# 运行主程序
!python main.py

print("="*60)
print("✓ 任务完成")
print(f"⏰ 完成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

## 6️⃣ 查看结果

In [ ]:
# 查看统计报告
with open('output/statistics_report.txt', 'r', encoding='utf-8') as f:
    print(f.read())

In [ ]:
# 查看概念列表
import pandas as pd

# 找到最新的triples文件
import glob
csv_files = glob.glob('output/triples/triples_*.csv')
if csv_files:
    latest_csv = max(csv_files)
    df = pd.read_csv(latest_csv)
    print(f"\n概念总数: {len(df)}\n")
    print("前20个概念:")
    print(df.head(20))
else:
    print("未找到结果文件")

In [ ]:
# 简单可视化
import matplotlib.pyplot as plt
import seaborn as sns

# 类别分布
if 'category' in df.columns:
    plt.figure(figsize=(12, 6))
    category_counts = df['category'].value_counts()
    sns.barplot(x=category_counts.values, y=category_counts.index)
    plt.title('概念类别分布', fontsize=16)
    plt.xlabel('数量', fontsize=12)
    plt.ylabel('类别', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print("\n类别统计:")
    print(category_counts)

## 7️⃣ 下载结果

In [ ]:
# 打包结果文件
import shutil
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_zip = f'PWD_KG_Results_{timestamp}.zip'

# 创建压缩包
shutil.make_archive(
    output_zip.replace('.zip', ''),
    'zip',
    'output'
)

print(f"✓ 结果已打包: {output_zip}")
print(f"  文件位置: {WORK_DIR}/{output_zip}")
print("\n可以从Google Drive下载，或使用下面的代码直接下载:")

# 直接下载到本地
from google.colab import files
files.download(output_zip)

## 8️⃣ 导入Neo4j (可选)

如果您有Neo4j数据库，可以导入生成的Cypher脚本

In [ ]:
# 查看Neo4j导入脚本
!ls -lh output/neo4j_import/

print("\n导入方法:")
print("1. 下载 output/neo4j_import/ 文件夹")
print("2. 在Neo4j Browser中运行 import.cypher")
print("3. 或使用 Python脚本: python output/neo4j_import/import_to_neo4j.py")

## 9️⃣ 清理 (可选)

In [ ]:
# 清理缓存文件 (保留结果)
!rm -rf cache/
!rm -rf __pycache__/

print("✓ 缓存已清理")
print("  结果文件保留在 output/ 目录")

---

## 💡 使用技巧

### 加快速度
```python
MAX_CHUNKS = 50  # 12.5分钟，快速测试
```

### 提高质量
```python
MAX_CHUNKS = 200  # 50分钟，更多概念
```

### 处理全部文本
```python
MAX_CHUNKS = None  # 80分钟，最完整
```

### 保持连接
- Colab免费版会在90分钟后断开
- 建议使用100-150块的配置
- 或升级到Colab Pro

### 监控进度
```python
!tail -f output/kg_builder.log
```

---

## 📊 预期结果

| 块数 | 时间 | 概念数 |
|------|------|--------|
| 50 | 12.5分钟 | 150-180 |
| 100 | 25分钟 | 200-250 |
| 150 | 37.5分钟 | 250-300 |
| 321 | 80分钟 | 400-600 |

---

## ❓ 常见问题

**Q: Colab断开连接怎么办？**
A: 结果已保存在Google Drive，可以继续下载

**Q: 如何查看运行进度？**
A: 查看单元格输出或运行 `!tail output/kg_builder.log`

**Q: 可以使用GPU吗？**
A: Ollama会自动使用可用的GPU，无需额外配置

**Q: 文件保存在哪里？**
A: Google Drive的 `MyDrive/PWD_KG/output/` 目录

---

**完成后记得下载结果文件！** 📥
